In [2]:
# import packages
import ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
import scipy
import neurokit2 as nk
import random
import pyhrv

In [7]:
# Feature extraction PPG features
def get_bvpfeatures(data, index):
    features_bvp = None

    df, info = nk.ppg_process(data, sampling_rate=100) # Process data before extraction

    features = nk.ppg_intervalrelated(df) # Features extraced with Neurokit2 function
    features['index'] = index
    return features

In [1]:
# Data preparation
def bvp_prep(data):
    data = data.fillna(method="ffill") # Fill the NaN values with last known value
    data = data.fillna(method="bfill") # Fill the NaN values with next known value
    data = data.drop(['HRV_LF','HRV_LFn','HRV_LFHF'], axis = 1) # These features will be dropped
    # These features will be set to 0 if NaN
    data[['HRV_HF', 'HRV_VHF', 'HRV_HFn', 'HRV_LnHF', 'HRV_MSE', 'HRV_CMSE', 'HRV_RCMSE']] = data[['HRV_HF', 'HRV_VHF', 'HRV_HFn', 'HRV_LnHF', 'HRV_MSE', 'HRV_CMSE', 'HRV_RCMSE']].replace(np.nan, 0) 
    data = data.dropna(axis='columns') # Drop the columns containing NaN
    features = data
    features.replace([np.inf, -np.inf], np.nan, inplace=True) # Replace infinity values with NaN
    features = features.dropna(axis='columns') # Drop the columns containing NaN
    return features

In [9]:
# Filter the measured data
def sensorPPGfilter(data):
    fs = 400 # sampling frequency
    ir_data = data.iloc[100:,[3]] # select PPG data

    #plt.figure(figsize=(20,4))
    #plt.plot(ir_data)
    # labels and titles
    #plt.title('Raw IR signal')
    #plt.xlabel('$Index$') 
    #plt.ylabel('$PPG$') 
    #plt.figure(figsize=(20,4))

    ir_data_clean = nk.ppg_clean(ir_data, method='nabian2018', sampling_rate=1000) # clean function Neurokit

    # Parameters 
    order = 4
    frequency = np.array([0.5,10]) # cutoff freq of 0.5 and 10 Hz
    sampling_rate = 400
    frequency = frequency / (sampling_rate / 2)  # Normalize frequency to Nyquist Frequency (Fs/2).

    # Filtering
    b, a = scipy.signal.butter(order, frequency, btype='band') # 4th order Butterworth bandpass filter
    ir_filt = scipy.signal.filtfilt(b, a, ir_data_clean, axis=0) # apply filter to data

    peaks_red, _ = find_peaks(np.array(ir_filt), height=0, distance=0.5*fs) # peak detection
    peaks_red_min, _ = find_peaks(-ir_filt, height=0, distance=0.5*fs) # onset detection

    ir_filt = scipy.signal.filtfilt(b, a, ir_data_clean, axis=0)[peaks_red_min[0]-1:peaks_red_min[-1]-1] # slice data from first peak to last peak
    peaks_red, _ = find_peaks(np.array(ir_filt), height=0, distance=0.5*fs)
    peaks_red_min, _ = find_peaks(-ir_filt, height=0, distance=0.5*fs)

    # plot
    #plt.figure(figsize=(20,4))
    #plt.plot((red_data-red_data.iloc[0,0]))
    #plt.plot(-ir_filt) # plot the BVP data
    #plt.plot(peaks_red, red_filt[peaks_red], "x")
    #plt.plot(peaks_red_min, red_filt[peaks_red_min], "x")
    # labels and titles
    #plt.title('Filtered IR signal')
    #plt.xlabel('$Index$') 
    #plt.ylabel('$PPG$') 
    return -ir_filt # return the BVP data